Q1

In [1]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
embeddings = embedding_model.encode(user_question)

In [4]:
first_value = embeddings[0]
print(first_value)

0.078222655


Q2

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

In [8]:
len(filtered_documents)

375

In [9]:
embeddings = []

for doc in filtered_documents: # puede tartar un poco
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

In [10]:
import numpy as np

In [11]:
X = np.array(embeddings)
print(X.shape)

(375, 768)


Q3

In [12]:
v = embedding_model.encode(user_question)

In [13]:
scores = X.dot(v)
print(np.max(scores))

0.6506573


Q4

In [14]:
from tqdm.auto import tqdm

In [15]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [16]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [18]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [19]:
search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)

def search_function(q):
    v = embedding_model.encode(q["question"])
    return search_engine.search(v, num_results=5)

In [20]:
evaluate(ground_truth, search_function)

100%|██████████| 1830/1830 [01:29<00:00, 20.46it/s]


{'hit_rate': 0.9398907103825137}

Q5

In [22]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768, # dimension of the embedding X
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions-homework"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework'})

In [23]:
for d, emb in zip(tqdm(filtered_documents), embeddings):
    d['question_text_vector'] = emb
    es_client.index(index=index_name, document=d)

100%|██████████| 375/375 [00:10<00:00, 34.36it/s]


In [24]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [25]:
user_question = "I just discovered the course. Can I still join it?"

In [27]:
results = elastic_search_knn('question_text_vector', v, 'machine-learning-zoomcamp')

In [29]:
first_item_id = results[0]['id']
print(first_item_id)

ee58a693


Q6

In [34]:
def question_text_vector_knn(q):
    question = q['question']
    question_vector = embedding_model.encode(question)
    return elastic_search_knn('question_text_vector', question_vector, 'machine-learning-zoomcamp')

In [35]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 1830/1830 [01:28<00:00, 20.65it/s]


{'hit_rate': 0.9398907103825137}